# <span style="color:darkblue"> Lecture 26 - Essentials of Merging in SQL </span>

<font size = "5">

In this lecture we will cover some additional topics <br>
regarding merging of datasets in SQL.

SQL in 100 seconds:

https://www.youtube.com/watch?v=zsjvFFKOm3c

Overview of SQL:

https://www.youtube.com/watch?v=27axs9dO7AE

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">

Import libraries

In [33]:
# psycogpg2 helps us process SQL commands to send to the server
# sqlalchemy facilitates establishing a connection with the server

import pandas as pd
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

<font size = "5">

Connect to SQL server

- In the default instructions we set <br>
the password to "12345" in windows and <br>
no password for Mac
- ADJUST code accordingly! 

In [34]:
# Use the connection details to your server
# These are the default settings.
# "postgresql" is a fixed argument
# If you have a different host,database, username, or password,
# change the corresponding connection details


url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = "12345")

connection = create_engine(url_server)


# <span style="color:darkblue"> II. SQL Review </span>

<font size = "5">

United States Congress dataset

- Bills and actions
- US congress members



<font size = "5">

Read datasets into Python

In [35]:
bills_actions       = pd.read_csv("data_raw/bills_actions.csv")
bills_subjects      = pd.read_csv("data_raw/bills_subjects.csv")
us_congress_members = pd.read_csv("data_raw/us_congress_members.csv")

<font size = "5">

Upload to SQL


In [36]:
bills_actions.to_sql('bills_actions',
               con = connection,
               if_exists='replace',
               index=False)

bills_subjects.to_sql('bills_subjects',
               con = connection,
               if_exists='replace',
               index=False)

us_congress_members.to_sql('us_congress_members', 
             con = connection, 
             if_exists='replace',
             index=False)

811

<font size = "5">

Download from SQL Server

In [37]:
# Recall that "*" indicates that we should select all columns

data  = pd.read_sql(text('SELECT * \
                          FROM bills_actions;'), connection)

display(data)

,congress,bill_number,bill_type,action,main_action,object,member_id
0,116,1029,s,S.Amdt.1274 Amendment SA 1274 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
1,116,1031,s,S.Amdt.2698 Amendment SA 2698 proposed by Sena...,senate amendment proposed (on the floor),amendment,675
2,116,1160,s,S.Amdt.2659 Amendment SA 2659 proposed by Sena...,senate amendment proposed (on the floor),amendment,858
3,116,1199,s,"Committee on Health, Education, Labor, and Pen...",senate committee/subcommittee actions,senate bill,1561
4,116,1208,s,Committee on the Judiciary. Reported by Senato...,senate committee/subcommittee actions,senate bill,1580
...,...,...,...,...,...,...,...
3298,116,9,hr,H.Amdt.172 Amendment (A004) offered by Ms. Kus...,house amendment offered,amendment,36
3299,116,9,hr,H.Amdt.171 Amendment (A003) offered by Ms. Hou...,house amendment offered,amendment,186
3300,116,9,hr,H.Amdt.170 Amendment (A002) offered by Ms. Oma...,house amendment offered,amendment,477
3301,116,9,hr,POSTPONED PROCEEDINGS - At the conclusion of d...,other house amendment actions,amendment,393


<font size = "5">

Try it yourself

Download and display the "us_congress_members" dataset from SQL

In [38]:
# Write your own code

data  = pd.read_sql(text('SELECT full_name, last_name \
                          FROM us_congress_members;'), connection)

display(data)



,full_name,last_name
0,A. Donald McEachin,McEachin
1,Aaron Schock,Schock
2,Abby Finkenauer,Finkenauer
3,Abigail Davis Spanberger,Spanberger
4,Adam H. Putnam,Putnam
...,...,...
1806,Wm. Lacy Clay,Clay
1807,"Wyche, Jr. Fowler",Fowler
1808,"Wyche, Jr. Fowler",Fowler
1809,Yvette D. Clarke,Clarke


# <span style="color:darkblue"> III. Merge Dataset </span>

<font size = "5">

The following is an example of an entity relationship diagram (ERD)



<font size = "5">

<img src="figures/rdb_us_congress.png" alt="drawing" width="650"/>

- Here "member_id" is the PRIMARY KEY of the first dataset
- We can check that with SQL commands

In [39]:
summary = pd.read_sql(text("SELECT COUNT(DISTINCT member_id) AS num_distinct, \
                           COUNT(*) AS num_members   \
                           FROM us_congress_members"),
                      connection)

summary

,num_distinct,num_members
0,1811,1811


<font size = "5">

Merge two datasets

- Similar to python
- FROM is followed by the name of the primary dataset
- LEFT is followed by the name of the secondary dataset
- ON is the id variable used for merging

In [40]:
# Write your own code

example = pd.read_sql(text("SELECT * \
                            FROM bills_actions \
                            LEFT JOIN us_congress_members \
                            ON bills_actions.member_id = us_congress_members.member_id ;"),
                      connection)

example


,congress,bill_number,bill_type,action,main_action,object,member_id,member_id,full_name,last_name,member_title,state,party_name,chamber
0,116,3,hr,H.Amdt.719 Amendment (A009) offered by Ms. Fin...,house amendment offered,amendment,2,2,Abby Finkenauer,Finkenauer,Representative,Iowa,Democratic,House
1,116,7617,hr,H.Amdt.870 Amendment (A012) offered by Ms. Fin...,house amendment offered,amendment,2,2,Abby Finkenauer,Finkenauer,Representative,Iowa,Democratic,House
2,116,1644,hr,H.Amdt.167 Amendment (A011) offered by Ms. Spa...,house amendment offered,amendment,3,3,Abigail Davis Spanberger,Spanberger,Representative,Virginia,Democratic,House
3,116,4617,hr,H.Amdt.651 Amendment (A014) offered by Ms. Spa...,house amendment offered,amendment,3,3,Abigail Davis Spanberger,Spanberger,Representative,Virginia,Democratic,House
4,116,3055,hr,H.Amdt.429 Amendment (A043) offered by Ms. Spa...,house amendment offered,amendment,3,3,Abigail Davis Spanberger,Spanberger,Representative,Virginia,Democratic,House
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3298,116,1749,s,Mr. Clay asked unanimous consent to discharge ...,house floor actions,senate bill,1806,1806,Wm. Lacy Clay,Clay,Representative,Missouri,Democratic,House
3299,116,4761,hr,Ms. Clarke (NY) moved to suspend the rules and...,house floor actions,house bill,1809,1809,Yvette D. Clarke,Clarke,Representative,New York,Democratic,House
3300,116,4739,hr,Ms. Clarke (NY) moved to suspend the rules and...,house floor actions,house bill,1809,1809,Yvette D. Clarke,Clarke,Representative,New York,Democratic,House
3301,116,3055,hr,H.Amdt.401 Amendment (A015) offered by Ms. Cla...,house amendment offered,amendment,1809,1809,Yvette D. Clarke,Clarke,Representative,New York,Democratic,House


# <span style="color:darkblue"> III. Examples of special cases </span>

<font size = "5">

Merging with multiple keys



In [ ]:
# Write your own code


example = pd.read_sql(text("SELECT * \
                            FROM bills_actions \
                            LEFT JOIN bills_subjects \
                            ON  bills_actions.bill_number = bills_subjects.bill_number \
                                AND bills_actions.bill_type = bills_subjects.bill_type;"),
                      connection)

example


,congress,bill_number,bill_type,action,main_action,object,member_id,congress,bill_number,bill_type,bill_subject
0,116,1,hjres,Ms. Granger moved to recommit to the Committee...,house committee/subcommittee actions,house joint resolution,693,116.0,1.0,hjres,Appropriations
1,116,1,hjres,Ms. Granger moved to recommit to the Committee...,house committee/subcommittee actions,house joint resolution,693,116.0,1.0,hjres,State and local government operations
2,116,1,hjres,Ms. Granger moved to recommit to the Committee...,house committee/subcommittee actions,house joint resolution,693,116.0,1.0,hjres,"Government employee pay, benefits, personnel m..."
3,116,1,hjres,Ms. Granger moved to recommit to the Committee...,house committee/subcommittee actions,house joint resolution,693,116.0,1.0,hjres,Immigration status and procedures
4,116,1,hjres,Ms. Granger moved to recommit to the Committee...,house committee/subcommittee actions,house joint resolution,693,116.0,1.0,hjres,State and local finance
...,...,...,...,...,...,...,...,...,...,...,...
154019,116,9051,hr,Mr. Neal moved to suspend the rules and pass t...,house floor actions,house bill,1671,116.0,9051.0,hr,Cardiovascular and respiratory health
154020,116,9051,hr,Mr. Neal moved to suspend the rules and pass t...,house floor actions,house bill,1671,116.0,9051.0,hr,Infectious and parasitic diseases
154021,116,9051,hr,Mr. Neal moved to suspend the rules and pass t...,house floor actions,house bill,1671,116.0,9051.0,hr,"Tax administration and collection, taxpayers"
154022,116,9051,hr,Mr. Neal moved to suspend the rules and pass t...,house floor actions,house bill,1671,116.0,9051.0,hr,Income tax credits


<font size = 5>
Subsetting specific columns of the secondary dataset before merging <br>
via "common table expressions"


In [44]:
# The first part of the code creates an 
# intermediate dataset (temporarily stored but deleted after the query)
# The rest of the code just merges the data.

example = pd.read_sql(text("WITH intermediate_dataset AS ( \
                                SELECT member_id, full_name \
                                FROM us_congress_members \
                            ) \
                            SELECT * \
                            FROM bills_actions \
                            LEFT JOIN intermediate_dataset \
                            ON bills_actions.member_id = intermediate_dataset.member_id ;"),
                      connection)

example


,congress,bill_number,bill_type,action,main_action,object,member_id,member_id,full_name
0,116,1029,s,S.Amdt.1274 Amendment SA 1274 proposed by Sena...,senate amendment proposed (on the floor),amendment,858,858,Mitch McConnell
1,116,1031,s,S.Amdt.2698 Amendment SA 2698 proposed by Sena...,senate amendment proposed (on the floor),amendment,675,675,Josh Hawley
2,116,1160,s,S.Amdt.2659 Amendment SA 2659 proposed by Sena...,senate amendment proposed (on the floor),amendment,858,858,Mitch McConnell
3,116,1199,s,"Committee on Health, Education, Labor, and Pen...",senate committee/subcommittee actions,senate bill,1561,1561,Lamar Alexander
4,116,1208,s,Committee on the Judiciary. Reported by Senato...,senate committee/subcommittee actions,senate bill,1580,1580,Lindsey Graham
...,...,...,...,...,...,...,...,...,...
3298,116,9,hr,H.Amdt.172 Amendment (A004) offered by Ms. Kus...,house amendment offered,amendment,36,36,Ann M. Kuster
3299,116,9,hr,H.Amdt.171 Amendment (A003) offered by Ms. Hou...,house amendment offered,amendment,186,186,Chrissy Houlahan
3300,116,9,hr,H.Amdt.170 Amendment (A002) offered by Ms. Oma...,house amendment offered,amendment,477,477,Ilhan Omar
3301,116,9,hr,POSTPONED PROCEEDINGS - At the conclusion of d...,other house amendment actions,amendment,393,393,"Frank, Jr. Pallone"


# <span style="color:darkblue"> V. (Optional) Additional Resources </span>


<font size = "5">


Introduction to databases:

https://www.youtube.com/watch?v=wR0jg0eQsZA

Introduction to Entity Relationship Diagrams

https://www.youtube.com/watch?v=wR0jg0eQsZA




<font size = 5>

Examples of advanced techniques

https://medium.com/@techsuneel99/20-advanced-sql-techniques-with-practical-examples-b47490d9896d


# <span style="color:darkblue"> VI. Concluding Remarks </span>


<font size = "5">

A few things to take away from this class:

(1) Modularity:

- Python is built over smaller components
- It depends on a notion of objects and functions applied <br>
to those objects.
- Emphasized "chaining" of operations.

(2) Specialization:

- Python is great at math, logic, and data analysis.
- SQL is great at updating, managing and retrieving datasets, <br>
particularly at a corporate level.
- Integrations across languages is straightforward!

(2) Extensibility

- We only covered a fraction of the things that Python can do.
- New functionalities rely on the basic principles covered in the class!
- Keep practicing new things! :)





# <span style="color:darkblue"> Thank you to our fantastic TAs! </span>




# <span style="color:darkblue"> V. Where to go from here?  </span>

<font size = "5">

More courses in QTM!

https://quantitative.emory.edu/undergraduate/courses.html

<font size = "5">

Data Camp

https://www.datacamp.com/


<font size = "5">

Course era:

https://www.coursera.org/specializations/python


<font size = "5">

Prepare for coding interviews:

https://leetcode.com/




# <span style="color:darkblue"> Thank you for a wonderful semester! </span>


